In [1]:
# Librerías
import requests
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from astroquery.mpc import MPC

In [2]:
# Conexión con la API de COBS
url = 'https://cobs.si/api/obs_list.api?des=C/2023 A3&format=json&from_date=&to_date=&exclude_faint=False&exclude_not_accurate=False'
response = requests.get(url) 

if response.status_code == 200:
    content = response.json()
    print('✅ Base de datos actualizada.')
else:
    print(f'🛑 Se presentó un error al cargar la base de datos.\nError: {response.status_code}\n{response.content}')

✅ Base de datos actualizada.


In [3]:
# Creación del data frame Cometa
cometa_df = pd.DataFrame(content['objects'])
cometa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 47 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   type                       1155 non-null   object 
 1   obs_date                   1155 non-null   object 
 2   comet                      1155 non-null   object 
 3   observer                   1155 non-null   object 
 4   location                   528 non-null    object 
 5   extinction                 68 non-null     object 
 6   obs_method                 1155 non-null   object 
 7   comet_visibility           3 non-null      object 
 8   magnitude                  1155 non-null   object 
 9   conditions                 18 non-null     object 
 10  ref_catalog                1155 non-null   object 
 11  instrument_aperture        1155 non-null   object 
 12  instrument_type            1155 non-null   object 
 13  instrument_focal_ratio     1012 non-null   float

In [4]:
# Sin filtrar la información
filas,columnas = cometa_df.shape
print(f'Registros: {filas}\nVariables: {columnas}')

Registros: 1155
Variables: 47


In [5]:
# Base de datos arrojada por la API
cometa_df.sample(5)

,type,obs_date,comet,observer,location,extinction,obs_method,comet_visibility,magnitude,conditions,...,magnitude_error,comparison_star_magnitude,pixel_size_x,pixel_size_y,pixel_size_unit,obs_comment,obs_sky_quality,obs_sky_quality_method,reference_star_names,date_added
275,C,2024-06-02 21:49:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Douglas', 'last_name': 'Heggie...","ASERO, Trevinca, Spain",None,"{'key': 'Z', 'name': 'CCD Visual equivalent', ...",None,10.7,None,...,None,None,None,None,None,Limiting magnitude 20.0,None,NaN,,2024-06-06 14:16:52
696,C,2024-04-27 21:47:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Douglas', 'last_name': 'Heggie...","Aldclune, Perthshire, UK",None,"{'key': 'Z', 'name': 'CCD Visual equivalent', ...",None,10.7,None,...,None,None,None,None,None,Limiting magnitude 17.1,None,NaN,,2024-04-30 20:03:46
858,C,2024-03-30 19:15:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Jiarui', 'last_name': 'Xiong',...","Daocheng, China(remote,O40)",None,"{'key': 'C', 'name': 'Unfiltered total CCD mag...",None,11.3,None,...,None,None,None,None,None,,None,NaN,,2024-03-31 08:24:41
953,C,2024-02-08 18:22:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Michael', 'last_name': 'Mattia...","Swan Hill, Victoria, Australia",None,"{'key': 'Z', 'name': 'CCD Visual equivalent', ...",None,13.7,None,...,None,None,None,None,None,,None,NaN,,2024-02-10 09:02:13
709,V,2024-04-26 12:03:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'John Paul', 'last_name': 'Pile...","San Rafael, Bulacan, Philippines",None,"{'key': 'M', 'name': 'Modified-Out method', 's...",None,10.3,None,...,None,None,None,None,None,,None,NaN,,2024-04-26 12:07:23


In [6]:
# Métodos de observación
cometa_df.obs_method.apply(lambda registro: f'{registro['key']}: {registro['name']}').value_counts()

obs_method
C: Unfiltered total CCD magnitude            341
Z: CCD Visual equivalent                     292
S: In-Out method                             193
M: Modified-Out method                       178
V: Johnson/Bessel/Kron/Cousins V with CCD     76
B: Simple Out-Out method                      19
k: Kron/Cousins R with CCD                    18
H: Kron/Cousins I with CCD                    17
D: Johnson/Bessel/Kron/Cousins B with CCD     10
c: Unfiltered nuclear CCD magnitude            8
d: Astrodon G filter                           2
E: Extrafocal-Extinction method                1
Name: count, dtype: int64

In [7]:
# Tratamiento de los datos de interés
cometa_df['obs_method_key'] = cometa_df.obs_method.apply(lambda registro: registro['key'])
cometa_df['obs_date'] = pd.to_datetime(pd.to_datetime(cometa_df.obs_date).dt.date)
cometa_df['magnitude'] = pd.to_numeric(cometa_df.magnitude)

In [8]:
# Creación del data frame curva de luz cruda
curva_de_luz_cruda_df = cometa_df[['obs_method_key', 'obs_date', 'magnitude']].copy()
curva_de_luz_cruda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   obs_method_key  1155 non-null   object        
 1   obs_date        1155 non-null   datetime64[ns]
 2   magnitude       1155 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 27.2+ KB


In [9]:
# Con la información filtrada
filas,columnas = curva_de_luz_cruda_df.shape
print(f'Registros: {filas}\nVariables: {columnas}')

Registros: 1155
Variables: 3


In [10]:
# Data Frame de la curva de luz
curva_de_luz_cruda_df.sample(5)

,obs_method_key,obs_date,magnitude
871,C,2024-03-20,12.1
188,S,2024-06-13,10.7
747,Z,2024-04-16,10.5
638,Z,2024-04-30,10.6
416,D,2024-05-14,11.4


In [11]:
# Curva de luz cruda.
labels = {'obs_date':'Observation Date','magnitude':'Apparent total magnitude', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_cruda_df, x='obs_date', y='magnitude', color='obs_method_key', template= 'plotly_dark', labels= labels, title='Lightcurve of comet C/2023 A3 (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [12]:
# Variables a graficar
tiempo = curva_de_luz_cruda_df.obs_date
obs_method = curva_de_luz_cruda_df.obs_method_key.unique()

# Inicializar la figura con subplots
fig = make_subplots(rows=4, cols=3, subplot_titles= obs_method)

# Agregar trazas a los subplots
for metodo, fila, columna  in zip(obs_method, [1]*3 + [2]*3 + [3]*3, [1,2,3]*3):
    fig.add_trace(go.Scatter(x=tiempo, y=curva_de_luz_cruda_df.magnitude[curva_de_luz_cruda_df.obs_method_key == metodo], mode="markers",name = metodo), row = columna, col = fila)

# Actualizar el diseño de los subplots
fig.update_layout(title_text = 'Lightcurve of comet C/2023 A3 (Tsuchinshan-ATLAS)', template = 'plotly_dark')
fig.update_yaxes(autorange="reversed")

# Mostrar la figura
fig.show()

In [13]:
# Creación de data frame Ephemeris (conexión con la API del MPC)
fecha_inicial = curva_de_luz_cruda_df.obs_date.min()
fecha_final = curva_de_luz_cruda_df.obs_date.max()
fechas = (fecha_final - fecha_inicial).days

ephemeris = MPC.get_ephemeris('C/2023 A3', start = str(fecha_inicial), number = fechas + 1) # type: ignore

ephemeris_df = ephemeris.to_pandas()
ephemeris_df.columns = ephemeris_df.columns.str.lower().str.replace(' ', '_')
ephemeris_df

,date,ra,dec,delta,r,elongation,phase,v,proper_motion,direction
0,2023-02-25,230.565833,-1.256389,6.946,7.290,106.6,7.5,19.1,8.17,318.9
1,2023-02-26,230.528333,-1.214722,6.921,7.281,107.6,7.4,19.1,8.51,317.4
2,2023-02-27,230.488750,-1.172778,6.896,7.272,108.6,7.4,19.1,8.86,316.0
3,2023-02-28,230.446667,-1.130000,6.871,7.264,109.6,7.4,19.1,9.21,314.7
4,2023-03-01,230.401667,-1.086389,6.847,7.255,110.6,7.3,19.1,9.56,313.4
...,...,...,...,...,...,...,...,...,...,...
530,2024-08-08,163.204167,-0.185000,2.008,1.234,29.7,24.0,10.2,21.97,220.4
531,2024-08-09,163.109583,-0.297222,2.003,1.216,28.7,23.6,10.2,22.06,219.7
532,2024-08-10,163.016250,-0.411111,1.997,1.198,27.8,23.3,10.1,22.18,219.1
533,2024-08-11,162.922917,-0.526667,1.991,1.180,26.9,22.9,10.1,22.31,218.6


In [14]:
# Info del data frame ephemeris
ephemeris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535 entries, 0 to 534
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           535 non-null    datetime64[ns]
 1   ra             535 non-null    float64       
 2   dec            535 non-null    float64       
 3   delta          535 non-null    float64       
 4   r              535 non-null    float64       
 5   elongation     535 non-null    float64       
 6   phase          535 non-null    float64       
 7   v              535 non-null    float64       
 8   proper_motion  535 non-null    float64       
 9   direction      535 non-null    float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 41.9 KB


In [15]:
# Dar a los datos el formato deseado
ephemeris_df.date = pd.to_datetime(ephemeris_df.date)
ephemeris_df.date = pd.to_datetime(ephemeris_df.date.dt.date)
ephemeris_df.dtypes

date             datetime64[ns]
ra                      float64
dec                     float64
delta                   float64
r                       float64
elongation              float64
phase                   float64
v                       float64
proper_motion           float64
direction               float64
dtype: object

In [16]:
# Creación del data frame ephemeris filtrada
ephemeris_filtrada_df = ephemeris_df[['date', 'delta','r', 'phase']].copy()
ephemeris_filtrada_df = ephemeris_filtrada_df.rename(columns = {'date':'obs_date'})
ephemeris_filtrada_df

,obs_date,delta,r,phase
0,2023-02-25,6.946,7.290,7.5
1,2023-02-26,6.921,7.281,7.4
2,2023-02-27,6.896,7.272,7.4
3,2023-02-28,6.871,7.264,7.4
4,2023-03-01,6.847,7.255,7.3
...,...,...,...,...
530,2024-08-08,2.008,1.234,24.0
531,2024-08-09,2.003,1.216,23.6
532,2024-08-10,1.997,1.198,23.3
533,2024-08-11,1.991,1.180,22.9


In [17]:
# Unión de las bases de datos COBS y MPC
curva_de_luz_procesada_df = curva_de_luz_cruda_df.merge(ephemeris_filtrada_df, on='obs_date')
curva_de_luz_procesada_df

,obs_method_key,obs_date,magnitude,delta,r,phase
0,Z,2024-08-12,8.2,1.985,1.161,22.5
1,M,2024-08-11,8.9,1.991,1.180,22.9
2,Z,2024-08-11,8.4,1.991,1.180,22.9
3,S,2024-08-10,9.0,1.997,1.198,23.3
4,M,2024-08-10,8.6,1.997,1.198,23.3
...,...,...,...,...,...,...
1150,C,2023-02-27,18.0,6.896,7.272,7.4
1151,C,2023-02-26,18.1,6.921,7.281,7.4
1152,C,2023-02-25,18.2,6.946,7.290,7.5
1153,C,2023-02-25,17.9,6.946,7.290,7.5


In [18]:
# Información del data frame curva de lus procesada
curva_de_luz_procesada_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   obs_method_key  1155 non-null   object        
 1   obs_date        1155 non-null   datetime64[ns]
 2   magnitude       1155 non-null   float64       
 3   delta           1155 non-null   float64       
 4   r               1155 non-null   float64       
 5   phase           1155 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 54.3+ KB


In [19]:
# Reducción de la magnitud aparente
beta = 0

curva_de_luz_procesada_df['magnitud_reducida'] = (
    curva_de_luz_cruda_df['magnitude'] 
    - 5 * np.log10(curva_de_luz_procesada_df['delta'] * curva_de_luz_procesada_df['r'])
    - (beta * curva_de_luz_procesada_df['phase'])
    )
curva_de_luz_procesada_df

,obs_method_key,obs_date,magnitude,delta,r,phase,magnitud_reducida
0,Z,2024-08-12,8.2,1.985,1.161,22.5,6.387036
1,M,2024-08-11,8.9,1.991,1.180,22.9,7.045234
2,Z,2024-08-11,8.4,1.991,1.180,22.9,6.545234
3,S,2024-08-10,9.0,1.997,1.198,23.3,7.105826
4,M,2024-08-10,8.6,1.997,1.198,23.3,6.705826
...,...,...,...,...,...,...,...
1150,C,2023-02-27,18.0,6.896,7.272,7.4,9.498744
1151,C,2023-02-26,18.1,6.921,7.281,7.4,9.588201
1152,C,2023-02-25,18.2,6.946,7.290,7.5,9.677688
1153,C,2023-02-25,17.9,6.946,7.290,7.5,9.377688


In [20]:
# Curva de luz reducida
labels = {'obs_date':'Observation Date','magnitud_reducida':'Apparent total magnitude processed', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_procesada_df, x='obs_date', y='magnitud_reducida', color='obs_method_key', template= 'plotly_dark', labels= labels, title='Lightcurve of comet C/2023 A3 (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [21]:
# Variables a graficar
tiempo = curva_de_luz_procesada_df.obs_date
obs_method = curva_de_luz_procesada_df.obs_method_key.unique()

# Inicializar la figura con subplots
fig = make_subplots(rows=4, cols=3, subplot_titles= obs_method)

# Agregar trazas a los subplots
for metodo, fila, columna  in zip(obs_method, [1]*3 + [2]*3 + [3]*3, [1,2,3]*3):
    fig.add_trace(go.Scatter(x=tiempo, y=curva_de_luz_procesada_df.magnitud_reducida[curva_de_luz_procesada_df.obs_method_key == metodo], mode="markers",name = metodo), row = columna, col = fila)

# Actualizar el diseño de los subplots
fig.update_layout(title_text = 'Processed lightcurve of comet C/2023 A3 (Tsuchinshan-ATLAS)', template = 'plotly_dark')
fig.update_yaxes(autorange="reversed")

# Mostrar la figura
fig.show()

In [22]:
# Creación del data frame curva de luz agrupada
curva_de_luz_agrupada_df = curva_de_luz_procesada_df.groupby(by = 'obs_date').max()
curva_de_luz_agrupada_df = curva_de_luz_agrupada_df.reset_index()
curva_de_luz_agrupada_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida
0,2023-02-25,C,18.3,6.946,7.290,7.5,9.777688
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201
2,2023-02-27,C,18.0,6.896,7.272,7.4,9.498744
3,2023-02-28,C,18.2,6.871,7.264,7.4,9.709021
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311
...,...,...,...,...,...,...,...
297,2024-08-07,M,8.7,2.013,1.252,24.4,6.692759
298,2024-08-09,C,8.7,2.003,1.216,23.6,6.766927
299,2024-08-10,Z,9.0,1.997,1.198,23.3,7.105826
300,2024-08-11,Z,8.9,1.991,1.180,22.9,7.045234


In [23]:
# Curva de luz reducida
labels = {'obs_date':'Observation Date','magnitud_reducida':'Max apparent total magnitude reduced', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_agrupada_df, x=curva_de_luz_agrupada_df.obs_date, y='magnitud_reducida', color='obs_method_key', template= 'plotly_dark', labels= labels, title='Lightcurve of comet C/2023 A3 (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [24]:
# Variables a graficar
tiempo = curva_de_luz_agrupada_df.obs_date
obs_method = curva_de_luz_agrupada_df.obs_method_key.unique()

# Inicializar la figura con subplots
fig = make_subplots(rows=4, cols=3, subplot_titles= obs_method)

# Agregar trazas a los subplots
for metodo, fila, columna  in zip(obs_method, [1]*3 + [2]*3 + [3]*3, [1,2,3]*3):
    fig.add_trace(go.Scatter(x=tiempo, y=curva_de_luz_agrupada_df.magnitud_reducida[curva_de_luz_agrupada_df.obs_method_key == metodo], mode="markers",name = metodo), row = columna, col = fila)

# Actualizar el diseño de los subplots
fig.update_layout(title_text = 'Processed lightcurve of comet C/2023 A3 (Tsuchinshan-ATLAS)', template = 'plotly_dark')
fig.update_yaxes(autorange="reversed")

# Mostrar la figura
fig.show()

In [25]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 7

curva_de_luz_promediada_df = curva_de_luz_agrupada_df.copy()
curva_de_luz_promediada_df['grupo'] = curva_de_luz_promediada_df.index // numero_elementos_grupo
curva_de_luz_promediada_df = curva_de_luz_promediada_df.groupby(by = 'grupo')[['obs_date','magnitud_reducida']].mean()
curva_de_luz_promediada_df.obs_date = pd.to_datetime(curva_de_luz_promediada_df.obs_date.dt.date)
curva_de_luz_promediada_df

,obs_date,magnitud_reducida
grupo,,
0,2023-02-28,9.609031
1,2023-03-19,9.735617
2,2023-04-01,8.819320
3,2023-04-17,8.988288
4,2023-04-26,9.037432
5,2023-05-08,8.944687
6,2023-05-19,9.013734
7,2023-05-27,9.115467
8,2023-06-04,8.926538


In [26]:
# Gráfica de lus promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_promediada_df, x=curva_de_luz_promediada_df.obs_date, y='magnitud_reducida', template= 'plotly_dark', labels= labels, title='Lightcurve of comet C/2023 A3 (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()